In [18]:
import os
import re
import requests
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd

In [19]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [20]:
kam = 'data_raw/nobelistvo'

In [21]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [22]:
koho = pd.read_json(os.path.join('data_raw','wikidata_nobelovky_pro_filtr.json'))

In [23]:
sloupce = ['100','245','700']

In [24]:
df = pd.read_parquet(os.path.join('data/cnb_sloupce','008.parquet'))
for s in sloupce:
    df = df.merge(pd.read_parquet(os.path.join('data/cnb_sloupce',f'{s}.parquet')), left_index=True, right_index=True, how='outer')

In [25]:
df.sample(10)

,008,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,245_v,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_n,700_k,700_m,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_g
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ck8601100,860507s1986 xr a u0|0 | cze,1,"Brychová, Vlasta",jx20051130008,[aut],None,None,None,None,None,1,0,Český jazyk pro 4. ročník zvláštní školy /,None,"Vlasta Brychová, Eliška Nekudová ; ilustrovala...",None,None,None,None,None,None,"[1, 1]","[Nekudová, Eliška, Robejšková, Blanka]","[aut, ill]",None,"[jx20041229013, jx20050826009]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
bk193206566,991118s1931 xr ger,1,"Heyret, M.",None,[aut],None,None,None,None,None,1,4,Das Marienkind :,Ein Gebetbüchlein für jugendliche Marienverehr...,Vom M. Heyret,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpg20041410116,040806s2004 xr nnn g kncze,1,"Lednická, Zuzana,",jo2003193237,[art],1974-,None,None,None,None,1,0,Colours of Ostrava,music festival : 9.-11.7.2004 Ostrava centrum ...,"design Zuzana Lednická, foto Adam Holý",None,None,[grafika] :,None,None,None,[1],"[Holý, Adam,]",[pht],[1974-2016],[jx20050110007],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
cpg19970780213,970603s1994 xr cze,1,"Mucha, Alfons,",jk01082370,[art],1860-1939,None,None,None,None,1,0,Podzim,Fall = Automne = Herbst /,Alfons Mucha,None,None,[grafika] =,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk20031247202,030625s2003 xr a e p 001 0 cze,1,"Synek, Miloslav,",jk01122248,[aut],1935-2022,None,None,None,None,1,0,Manažerská ekonomika /,None,Miloslav Synek a kolektiv,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20203231566,200619s2020 xr a e 001 0 cze,1,"Vácha, Jiří,",jk01141116,[aut],1938-,None,None,None,None,1,0,Meze darwinismu /,None,Jiří Vácha,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk193606861,990921s1936 xr cze,1,"Zeyer, Julius,",jk01152552,[aut],1841-1901,None,None,None,None,1,0,Maeldunova výprava a jiné povídky /,None,Julius Zeyer,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpg20081788833,080125s2007 xr nnn g kncze,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Moravský podzim,42. mezinárodní hudební festival Brno : 21. zá...,None,None,None,[grafika] :,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk196101630,980311s1961 xr a 0|0 0 cze,1,"Albert, František,",jk01010154,[aut],1856-1923,None,None,None,None,1,0,Humanita /,None,"František Albert ; [Výbor] připr., [úv.] Život...",None,None,None,None,None,None,[1],"[Píč, Josef,]",[com],[1887-],[jk01092996],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [26]:
for index, row in koho.iterrows():
    nazev_souboru = f"nobelovka_{row['nobelovka']}_{row['100_7']}.parquet"
    if nazev_souboru not in os.listdir(kam):
        print(row['nobelovka'])
        df_slice = df[df['100_7'].astype(str).str.contains(row['100_7'], na=False) | df['700_7'].astype(str).str.contains(row['100_7'], na=False)]
        df_slice = df_slice.explode('008')
        df_slice['rok'] = df_slice['008'].apply(lambda x: najdi_rok(x))
        df_slice['nobelovka'] = row['nobelovka']
        df_slice['titul'] = df_slice['245_a'].apply(lambda x: x.replace(' /','').replace('=','').replace(' :','').replace(';','').strip() if x != None else None)
        df_slice.to_parquet(os.path.join(kam, nazev_souboru))